<a href="https://colab.research.google.com/github/alibabava2003/OCR_Vietnamese/blob/main/data_preparation/AUGUMENT_DATA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install albumentations opencv-python numpy


In [ ]:
import os
import cv2
import numpy as np
import albumentations as A

# Thư mục chứa ảnh gốc và ảnh augmented
input_folder = "/content/drive/MyDrive/dataaaaaa/OCR_dataset/Line/images"
output_folder = "/content/images4"
label_file = "/content/drive/MyDrive/dataaaaaa/OCR_dataset/Line/labels.txt"
output_label_file = "/content/labels4.txt"

os.makedirs(output_folder, exist_ok=True)

# Số ảnh augment mỗi ảnh gốc
num_augmentations = 5

# Hàm thêm padding lớn trước khi augment
def add_padding(image, pad=30):
    return cv2.copyMakeBorder(image, pad, pad, pad, pad, cv2.BORDER_CONSTANT, value=(255, 255, 255))

# Hàm cắt lại phần chứa chữ (loại bỏ khoảng trắng thừa)
def crop_to_content(image, threshold=240):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY_INV)
    coords = cv2.findNonZero(thresh)
    if coords is not None:
        x, y, w, h = cv2.boundingRect(coords)
        return image[y:y+h, x:x+w]
    return image

# Tạo pipeline augment
transform = A.Compose([
    A.Rotate(limit=8, border_mode=cv2.BORDER_CONSTANT, value=(255, 255, 255), p=0.7),
    A.Affine(shear={"x": (-10, 10), "y": (-10, 10)}, p=0.5),
    A.GaussNoise(p=0.5),
    A.MotionBlur(blur_limit=3, p=0.4),
    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.5),
    A.ToGray(p=0.3),
    A.Perspective(scale=(0.02, 0.08), p=0.3),
    A.GridDistortion(num_steps=5, distort_limit=0.3, p=0.3),
    A.Downscale(scale_min=0.6, scale_max=0.9, p=0.2),
])

# Đọc file label gốc
with open(label_file, "r", encoding="utf-8") as file:
    lines = file.readlines()

# Tạo file label mới
with open(output_label_file, "w", encoding="utf-8") as file:
    for line in lines:
        img_name, text = line.strip().split("\t", 1)
        img_path = os.path.join(input_folder, img_name)

        # Ghi tên file gốc vào file mới
        file.write(f"{img_name}\t{text}\n")

        # Đọc ảnh gốc
        image = cv2.imread(img_path)
        if image is None:
            continue

        # Thêm padding trước khi augment
        image = add_padding(image, pad=50)  # 🛠 Tăng padding lớn hơn

        # Tạo nhiều ảnh augment
        for i in range(num_augmentations):
            augmented = transform(image=image)["image"]

            # Cắt lại vùng chứa chữ
            augmented = crop_to_content(augmented)

            # Lưu ảnh augmented
            aug_img_name = f"aug_{i}_{img_name}"
            aug_img_path = os.path.join(output_folder, aug_img_name)
            cv2.imwrite(aug_img_path, augmented)

            # Ghi tên ảnh augmented vào file label mới
            file.write(f"{aug_img_name}\t{text}\n")

print(f"✅ Hoàn tất tạo {num_augmentations} ảnh augment mỗi ảnh gốc! 🎉")


<ipython-input-9-783a49352d8e>:33: UserWarning: Argument(s) 'value' are not valid for transform Rotate
  A.Rotate(limit=8, border_mode=cv2.BORDER_CONSTANT, value=(255, 255, 255), p=0.7),
<ipython-input-9-783a49352d8e>:41: UserWarning: Argument(s) 'scale_min, scale_max' are not valid for transform Downscale
  A.Downscale(scale_min=0.6, scale_max=0.9, p=0.2),


✅ Hoàn tất tạo 5 ảnh augment mỗi ảnh gốc! 🎉
